In [1]:
%load_ext autotime

In [2]:
import os
from glob import glob

from astropy.io import fits

from tqdm import tqdm_notebook

from piaa.utils import pipeline
from pocs.utils.db import postgres as clouddb

time: 1.83 s


In [3]:
tess_pass = getpass()

# meta_cursor = clouddb.get_cursor(port=5433, db_pass=meta_pass, db_name='panoptes', db_user='postgres')
tess_cursor = clouddb.get_cursor(port=5432, db_pass=tess_pass, db_name='v6', db_user='postgres')

 ·············


time: 4.57 s


In [4]:
base_dir = '/var/panoptes/images/fields/PAN001/Hd189733/14d3bd/20180913T054325'

time: 1.02 ms


In [5]:
# Get all the files that start with a time stamp (skip pointing files)
fits_files = sorted(glob(
    os.path.join(base_dir, '*2018*.fits')
), )
print("Found {} FITS files".format(len(fits_files)))

Found 127 FITS files
time: 5.9 ms


In [8]:
# Lookup the point sources
point_sources = pipeline.lookup_point_sources(
    fits_files[0], 
    force_new=True,
    cursor=tess_cursor,
#     method='photutils',
#     catalog_match=False
)

time: 8 s


In [9]:
point_sources.head()

,x,y,ra,dec,background,flux_auto,flux_max,fluxerr_auto,fwhm,flags,snr,twomass,tmag,vmag,d2d
id,,,,,,,,,,,,,,,
10668226,85.6097,2472.1506,297.805148,29.917162,2069.734,863.4516,51.17600,140.79750,5.93,0,6.132578,19511458+2954592,12.481,15.109000,0.004925
10668271,89.4770,2463.4978,297.833667,29.906753,2070.347,338.5105,46.64364,87.05955,4.31,0,3.888264,19511929+2954427,11.918,12.448000,0.005819
10668594,96.2462,2474.6409,297.797523,29.887056,2068.997,483.9258,53.98406,83.23398,3.41,0,5.814041,19511193+2952460,10.944,14.108000,0.007895
10668747,104.3185,2471.7134,297.807503,29.864477,2068.759,1555.3280,156.24100,106.08520,4.52,0,14.661121,19511375+2952085,11.807,16.014999,0.004578
10669478,125.1726,2493.1638,297.738369,29.804486,2067.137,1228.4580,119.93320,116.66330,3.39,0,10.529944,19505667+2948367,11.226,11.573000,0.005967


time: 32.2 ms


In [ ]:
d0 = fits.getdata(fn).astype(np.int16) - camera_bias
wcs = WCS(fn)

star_data = list()

for i, star_row in tqdm_notebook(enumerate(point_sources.itertuples())
                                total=len(point_sources)
                                ):

    ticid = str(star_row.Index)

    for fits_file in fits_files:    
    
        # Lookup the point sources
        point_sources = pipeline.lookup_point_sources(
            fits_file, 
            force_new=True,
            cursor=tess_cursor,
        )

        star_positions = wcs.all_world2pix(point_sources.ra.values, point_sources.dec.values, 1)
        x_pos = star_positions[0][i]
        y_pos = star_positions[1][i]

        star_data.append((ticid, 'position', 'x', 0, x_pos))
        star_data.append((ticid, 'position', 'y', 0, y_pos))

        slice0 = helpers.get_stamp_slice(x_pos, y_pos, stamp_size)    
    
        with fits.open(fits_file) as hdul:
            ext = 0
            hdu = hdul[ext]
            header = hdu.header
            d0 = hdu.data
    
    d1 = d0[slice0].flatten()

    rgb_data = helpers.get_rgb_data(d1.reshape(*stamp_size), separate_green=True)

    for color, stamp in zip('rgcb', rgb_data):
        d2 = stamp.data.flatten()
        m0 = stamp.mask.flatten()
        for pixel_idx in range(len(d2)):
            pixel_value = d2[pixel_idx]
            is_masked = m0[pixel_idx]
            if not is_masked:
                star_data.append((
                    ticid,
                    'flux',
                    color,
                    pixel_idx,
                    pixel_value
                ))                 

csv_fn = os.path.join(
    base_dir,
    'stamps_{:02d}.csv'.format(frame_idx)
)

# Save to CSV
frame_table = Table(
    rows=star_data, 
    names=('ticid', 'category', 'label', 'position', 'value'),
    meta={
        'date_obs': date_obs,
        'wcs': wcs.to_header_string(relax=True)
    }
)
ascii.write(frame_table, output=csv_fn, format='ecsv', overwrite=True)